In [2]:
import os
import sys
parent_dir = "/Volumes/work/phd/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [3]:
from mat4py import loadmat
import scipy.io as sio
import numpy as np
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)
import numpy as np
path='/local_storage/datasets/farzaneh/Olfaction/fmri_test/'
from utils.ridge_tools import *
import pandas as pd
import ast
base_directory = ''
from sklearn.model_selection import cross_validate,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, mean_squared_error
import scipy

In [4]:
base_dir = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/'
base_dir = base_dir= '../../../../T5 EVO/fmri/csvs'

In [25]:
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/supporting_files/NEMO_s01/Fullmet_FIR.mat'
data_original = sio.loadmat(parent_input_sagar_original)

In [26]:
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/odor_responses_S1-3_regionized/odor_responses_S1.mat'
data1 = sio.loadmat(parent_input_sagar_original)
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/odor_responses_S1-3_regionized/odor_responses_S2.mat'
data2 = sio.loadmat(parent_input_sagar_original)
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/odor_responses_S1-3_regionized/odor_responses_S3.mat'
data3 = sio.loadmat(parent_input_sagar_original)

In [8]:
rois_all=  []
for data_s in [data1,data2,data3]:
    rois = {}
    rois['PirF']= data_s['odor_vals'][0][0]
    rois['PirT']= data_s['odor_vals'][0][1]
    rois['AMY']= data_s['odor_vals'][0][2]
    rois['OFC']= data_s['odor_vals'][0][3]
    rois_all.append(rois)


In [9]:
for rois in rois_all:
    for key in rois.keys():
        roi=rois[key]
        roi = np.moveaxis(roi,-1,0)
        # roi = np.mean(roi,-1)
        rois[key]=roi

In [13]:
def prepare_dataset(ds):
    # ds['y'] = ds['y'].apply(ast.literal_eval)
    ds['embeddings'] = ds['embeddings'].apply(ast.literal_eval)
    return ds

In [14]:
def custom_linear_regression(X,y):
    clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])    
    estimator= clf.fit(X,y)
    return estimator

In [18]:
parent_input_sagar = base_dir+'curated_datasets/mols_datasets/curated_sagar_subjects_nonaminus.csv'
sagar_mols = pd.read_csv(parent_input_sagar)


In [39]:
def explainable_variance(data, bias_correction=True, do_zscore=True):
    """Compute explainable variance for a set of voxels.

    Parameters
    ----------
    data : array of shape (n_repeats, n_times, n_voxels)
        fMRI responses of the repeated test set.
    bias_correction: bool
        Perform bias correction based on the number of repetitions.
    do_zscore: bool
        z-score the data in time. Only set to False if your data time courses
        are already z-scored.

    Returns
    -------
    ev : array of shape (n_voxels, )
        Explainable variance per voxel.
    """
    if do_zscore:
        data = scipy.stats.zscore(data, axis=1)

    mean_var = data.var(axis=1, dtype=np.float64, ddof=1).mean(axis=0)
    var_mean = data.mean(axis=0).var(axis=0, dtype=np.float64, ddof=1)
    ev = var_mean / mean_var

    if bias_correction:
        n_repeats = data.shape[0]
        ev = ev - (1 - ev) / (n_repeats - 1)
    return ev


In [27]:
# sagar_mols1 = sagar_mols.loc[sagar_mols['subject']==1].sort_values('cid')
# sagar_mols2 = sagar_mols.loc[sagar_mols['subject']==2].sort_values('cid')
# sagar_mols3 = sagar_mols.loc[sagar_mols['subject']==3].sort_values('cid')

sagar_mols1 = sagar_mols.loc[sagar_mols['subject']==1]
sagar_mols2 = sagar_mols.loc[sagar_mols['subject']==2]
sagar_mols3 = sagar_mols.loc[sagar_mols['subject']==3]

In [ ]:
fmri_data_y=rois_all[:]['PirF']
# fmri_data_y = np.expand_dims(fmri_data_y.mean(-1),0)
print("(n_repeats, n_samples_test, n_voxels) =", fmri_data_y.shape)
ev = explainable_variance(fmri_data_y)
print("(n_voxels,) =", ev.shape)

In [22]:
def pipeline_regression(X,y,times,n_components=None):
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    for i in range(times):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed+i)  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        if n_components is not None:
            pca = PCA(n_components=n_components)
            X_train=pca.fit_transform(X_train)
            X_test=pca.transform(X_test)
        linreg =custom_linear_regression(X_train,y_train)
        mseerrors, correlations=metrics_per_descritor(X_test,y_test,linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
    return mserrorrs_corssvalidated, correlations_corssvalidated

In [23]:
def metrics_per_descritor(X,y,linreg):
    predicted=linreg.predict(X)
    mseerrors = []
    correlations = []
    if len(y.shape)>1:
        for i in range(y.shape[1]):
            mseerror=mean_squared_error(predicted[:,i],y[:,i])
            correlation=scipy.stats.pearsonr(predicted[:,i], y[:,i])
            mseerrors.append(mseerror)
            correlations.append(correlation)
            # print(predicted[:,i], y[:,i])
        
    else:
        mseerror=mean_squared_error(predicted,y)
        correlation=scipy.stats.pearsonr(predicted, y)
        mseerrors.append(mseerror)
        correlations.append(correlation)
    return mseerrors, correlations
        # plot()
        
    
    

In [24]:
def compute_correlation_keller(subject,roi,times , n_components,per_descritor=False):
            
    fmri_data_y=rois_all[subject-1][roi]

    
    corrs_molformer=[]

    mses_molformer=[]
    fmri_data_y_mean = fmri_data_y.mean(-1)
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
    # for i in [0,1,2,3,4,13]:
        df_keller_mse_molformers=[]
        df_keller_cor_molformers=[]
        for y_i in range(fmri_data_y_mean.shape[1]): 
        # for y_i in range(7): 
            input_file_keller_molformer = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/sagar_molformer_embeddings_'+str(i)+'_Apr17.csv'
            df_embeddings = pd.read_csv(input_file_keller_molformer)
            # df_embeddings_group = average_sagar_subject(df_embeddings)
            df_embeddings = prepare_dataset(df_embeddings)
            embeddings=df_embeddings[df_embeddings['subject']==subject]['embeddings']
            embeddings  = np.asarray(embeddings.values.tolist())
            
            
            df_keller_mse_molformer, df_keller_cor_molformer = pipeline_regression(embeddings,fmri_data_y_mean[:,y_i],times,n_components=n_components)
            df_keller_mse_molformers.append(df_keller_mse_molformer)
            df_keller_cor_molformers.append(df_keller_cor_molformer)
        
    corrs_molformer.append(df_keller_cor_molformers)
    mses_molformer.append(df_keller_mse_molformers)

    return corrs_molformer,mses_molformer

In [ ]:
seed=2024

# def extract_x_y(subject,area,input)\
corrs_molfomers=[]
mses_molformers=[]
results_df_all = pd.DataFrame()

for subject in [2,3]:
    for roi in ['OFC']:
        corrs_molfomer,mses_molformer = compute_correlation_keller(subject=subject,roi=roi,times=30 ,n_components=20,per_descritor=False)
        corrs_molfomer_squeezed = np.squeeze(np.asarray(corrs_molfomer),0)
        corrs_molfomer_squeezed = np.squeeze(corrs_molfomer_squeezed,-2)
        
        mses_molfomer_squeezed = np.squeeze(np.asarray(mses_molformer),0)
        mses_molfomer_squeezed = np.squeeze(mses_molfomer_squeezed,-1)

        statistic= corrs_molfomer_squeezed[:,:,0]
        p_value= corrs_molfomer_squeezed[:,:,1]
        p_value_df = pd.DataFrame.from_dict(p_value)
        p_value_df['fmri_index'] = p_value_df.index
        p_value_df['data'] = "p_value"
        statistics_df = pd.DataFrame.from_dict(statistic)
        statistics_df['fmri_index'] = statistics_df.index
        statistics_df['data'] = "statistic"
        mses_df = pd.DataFrame.from_dict(mses_molfomer_squeezed)
        mses_df['fmri_index'] = mses_df.index
        mses_df['data'] = "mse"
        results_df = pd.concat([p_value_df, statistics_df])
        results_df = pd.concat([results_df, mses_df])
        results_df["roi"] = roi
        results_df["subject"] = subject
        results_df.to_csv('fmri_'+str(subject)+'_'+roi+'.csv', index=False)  
        results_df_all = pd.concat([results_df, results_df_all])
        

In [ ]:
results_df_all.to_csv('fmri_all.csv', index=False)  

In [ ]:
# corrs_molfomer_squeezed = np.squeeze(np.asarray(corrs_molfomer),0)
# corrs_molfomer_squeezed = np.squeeze(corrs_molfomer_squeezed,-2)
#  # corrs_molfomers.append(corrs_molfomer_squeezed)   

# mses_molfomer_squeezed = np.squeeze(np.asarray(mses_molformer),0)
# mses_molfomer_squeezed = np.squeeze(mses_molfomer_squeezed,-1)

# data_dict = {'mse': mses_molfomer_squeezed , "corr": corrs_molfomer_squeezed}
# data_df = pd.DataFrame.from_dict(data_dict)
# data_df.to_csv(roi +"_"+ str(subject)+ "fmri.csv")

In [ ]:
np.asarray(mses_molformer).shape 
